# 1. Generate	per	base	call	distribution	of	quality	scores	for	read1,	read2,	index1,	and	index2

### a. Turn	in	the	8 histograms.

All plots are in the plots html file.

### b. What	is	a	good	quality	score	cutoff for	index	reads	and	pairs	to	utilize	for	sample identification	and	downstream	analysis,	respectively?

For index reads I proposed a cutoff quality score of 30. For both data sets, exluding 38, 37 is the highest quality score observed throughout all the reads. Beyond that the quality seems to slowly taper down linearly, the lower the score the fewer the reads belonging to that group. There are a few exceptions, such as 35 having fewer reads than 34, however the categories are all fairly close together until 30. In both data sets the data begins to vary drastically between a quality score of 24 and 29. With a quality score of 30 we still retain over 90% of our data and eliminate some of the unwanted variance due to errors.

For sequence reads I would select between a quality score of 35 or 30 depending on how much specificity I was looking for. For higher quality data I would use a cutoff score of 35, as that contains the 6 largest QS bins for both our FW and RV reads and also comprises over 90% of the data. However if looking for rare variants, or aligning to a good reference then I also thin ka QS cutoff of 30 may be acceptable. Likewise it would include the top 11 bin categories and add a 30-50~ million more reads. Ultimately it depends on how much data you have generated, though when in doubt it is better to sacrifice quantity for quality.

### c. How	many	indexes	have	Undetermined	(N)	base	calls? (Utilize	your	command	line	tool	knowledge.	Summit	the	command	you	used.	CHALLENGE:	use	a	one line	command)

Total Reads: 363246735

For the first index I used a simple cat + awk for sequence lines plus 'N': 3976613, which is just over 1%

```
cat 1294_S1_L008_R2_001.fastq | awk 'NR%4==2' | grep "N" | wc -l
```

Same for the second index: 3328051, which is just under 1%

```
cat 1294_S1_L008_R3_001.fastq | awk 'NR%4==2' | grep "N" | wc -l
```

# 2. Write	a	program	to	de-multiplex	the	samples	and	document	index	swapping and number	of	reads	retained	per	sample.

### a. How	many reads	are	retained	for	each	expected	index	pair?	What	is	the	percentage?

See table on HTML for all numbers/percents.

In total we had over 93.42% of our reads kept after our quality cutoff of 30, and of those, 94.9% were proper index pairs. However of the 24 indexes the top 3 indexes by count make up over 45% of the sequences. This means we had a large amount of preferential sequencing or amplification, specially for our TACCGGAT index which represented 23.02% of our sample.

### b. How	many	reads	are	indicative	of	index	swapping?

Total number of reads  kept: 339335851
From our retained reads, we observed index swapping without and undefined character and being a perfect match for our original indexes, on 606628 reads. This amounts to about 0.18% of our quality trimmed indexes.

However if you combine that with index pairs with at least one N (.99%) and index pairs that don't match our original indexes (3.93%, likely sequence errors) then in total we observed about 5% unexpected indexing results in our quality trimmed indexes.

### c. Create	a	distribution	of	swapped	indexes.	What	does	this	histogram	tell	you/what	is	your	interpretation	of	this	data?

In our case it is clear that index swapping is an event that is not entirely random. We observed a clear favoring of index TATGGCAC and TGTTCCGT being preferentially swapped with both combinations of those indexes representing over 25% of the 522 observed swaps. This is surprising as these samples' abundance on our proper index matches were only 5th and 8th, representing 4.76% and 3.36% of indexes, hardly enough to warrant over 25% of swaps. There is clearly another relationship at play. This is further evidenced by the fact that both of these indexes are not present in any other combination in the rest of the top 10 observed swaps.

According to our index file, these two indexes belong samples 17 & 19, groups 3E and 3F both the fox treatment. This may have something to do with the preverential swap. However the most likely explanation which is supported by the research is that these samples had a higher number of index library added to the sample which did not entirely bind, and that was then not accurately size selected.

In colusion there seem to be a select few (in this case two) indexes that showed a large degree of index swapping. Though they equate to the same treatment and subsequent samples, this is likely a coincidence and the real reason for this over-representation was a large amount of leftover index library leftover during sample preparation.

The table of all index swaps observed sorted by abundance is located at the end of the html file.

# The Cell Following this one includes the code to generate all the quality scores

In [ ]:
#Lane Hop Assignment

# R1 = Forward read file
# R2 = First index
# R3 = Reverse index
# R4 = Reverse read

#To convert phred scores to numbers
def convert_phred(letter):
    """Converts a single character into a phred score"""
    QScore = ord(letter) - 33
    return QScore

#First we get all the quality scores for both reads and bp positions

###################################################################################################

#First we create an array that will hold all the scores for each bp position of the read, and define it all as a function
#Additionally it will also create a dictionary with the QS of each read rounded out
def Get_QS(file):
    myfile = open(file)
    readQS = {}
    mean_scores = []
    NR = 0
    for lines in myfile:
        lines=lines.rstrip()
        current_read = 0
        if NR == 1:
            for i in range(len(lines)):
                mean_scores.append(0.0)
        NR = NR + 1
        #This first line will simply add the score for the first time through, makes it easier to divide by two further on
        if NR == 4:
            for i in range((len(lines))):
                #In this part we are using current_read to find the QS for the whole read, and mean_scores for the bp QS
                current_read = current_read + convert_phred(lines[i])
                mean_scores[i] = mean_scores[i] + convert_phred(lines[i])
            #Here we are rounding our average read QS and then placing it in our dictionary
            current_read = round(current_read/((len(lines))))
            if current_read in readQS:
                readQS[current_read] +=1
            else:
                readQS[current_read] = 1
        elif NR%4 == 0: #This selects the quality score lines, this bottom is a repeat of the top except for =! first quality line
            for i in range((len(lines))):
                current_read = current_read + convert_phred(lines[i])
                mean_scores[i] = mean_scores[i] + convert_phred(lines[i])
            current_read = round(current_read/(len(lines)))
            if current_read in readQS:
                readQS[current_read] +=1
            else:
                readQS[current_read] = 1
    for each in range(len(mean_scores)):
        mean_scores[each]=round((mean_scores[each]/(NR/4)),2)
    myfile.close()
    print("NR IS",NR)
    return [readQS,mean_scores] #Output is a list, with the first item being our dictionary of QS per read, and the second
                                #item being a list of our bp position QS

#Run out function on all 3 files to get both read QS and bp QS
R1QS = Get_QS("Undetermined_S0_R1_001_trunc.fastq")
R1readQS = R1QS[0]
R1bpQS = R1QS[1]

R2QS = Get_QS("Undetermined_S0_R2_001_trunc.fastq")
R2readQS = R2QS[0]
R2bpQS = R2QS[1]

R3QS = Get_QS("Undetermined_S0_R3_001_trunc.fastq")
R3readQS = R3QS[0]
R3bpQS = R3QS[1]

R4QS = Get_QS("Undetermined_S0_R4_001_trunc.fastq")
R4readQS = R4QS[0]
R4bpQS = R4QS[1]

#Get a final bp average QS for R1 and R4, this is the sequencing QS average per BP
FinalBPQS=[]
for i in range(101):
    FinalBPQS.append(0.0)
for i in range((len(R1bpQS))):
    FinalBPQS[i]=(R1bpQS[i]+R4bpQS[i])/2
    
#In this part we will combine all 4 dictionaries to a per read QS total
FinalreadQS={}
#Dic1
for key, value in R1readQS.items():
    if key in FinalreadQS:
        FinalreadQS[key] +=value
    else:
        FinalreadQS[key] = value
#Dic2
for key, value in R2readQS.items():
    if key in FinalreadQS:
        FinalreadQS[key] +=value
    else:
        FinalreadQS[key] = value
#Dic3
for key, value in R3readQS.items():
    if key in FinalreadQS:
        FinalreadQS[key] +=value
    else:
        FinalreadQS[key] = value
#Dic4        
for key, value in R4readQS.items():
    if key in FinalreadQS:
        FinalreadQS[key] +=value
    else:
        FinalreadQS[key] = value

###############################################################################
#Write out our results to separate files

#First one file for each original base pair QS
with open('R1readQS.txt', 'w') as f:
     f.write(str(R1readQS))
with open('R2readQS.txt', 'w') as f:
     f.write(str(R2readQS))
with open('R3readQS.txt', 'w') as f:
     f.write(str(R3readQS))
with open('R4readQS.txt', 'w') as f:
     f.write(str(R4readQS))

# #Next one file for each original read QS
with open('R1bpQS.txt', 'w') as f:
     f.write(str(R1bpQS))
with open('R2bpQS.txt', 'w') as f:
     f.write(str(R2bpQS))
with open('R3bpQS.txt', 'w') as f:
     f.write(str(R3bpQS))
with open('R4bpQS.txt', 'w') as f:
     f.write(str(R4bpQS))

#Finally a file for the combined read QS and bp QS
with open('FinalReadQS.txt', 'w') as f:
     f.write(str(FinalreadQS))
with open('FinalBPQS.txt', 'w') as f:
     f.write(str(FinalBPQS))

# The cell below includes the code to generate the preliminary Index Hop Data, no mean quality score used (for curiosity while the other ran)

In [ ]:
#Index Hopping Problem

#Write a function to get the complement of the reverse index
def Complement(index):
    #First we make our index into a list so we can modify it
    index = list(index)
    for each in range(len(index)):
        #First we replace all the nucleotides to complements
        if index[each]=='N':
            index[each]='N'
        elif index[each]=='A':
            index[each]='T'
        elif index[each]=='T':
            index[each]='A'
        elif index[each]=='C':
            index[each]='G'
        elif index[each]=='G':
            index[each]='C'
    #Now we return it to a string
    index="".join(index)
    #Then we inverse the order
    index=index[::-1]
    return index

#Establish our list of known indexes. THE FIRST IS THE REAL FILE, SECOND IS FOR TEST FILE, AND REVERSE COMPLEMENT
Indexes = ['GTAGCGTA', 'CGATCGAT', 'GATCAAGG', 'AACAGCGA', 'TAGCCATG', 'CGGTAATC', 'CTCTGGAT', 'TACCGGAT', 'CTAGCTCA', 'CACTTCAC', 'GCTACTCT', 'ACGATCAG', 'TATGGCAC', 'TGTTCCGT', 'GTCCTAAG', 'TCGACAAG', 'TCTTCGAC', 'ATCATGCG', 'ATCGTGGT', 'TCGAGAGT', 'TCGGATTC', 'GATCTTGC', 'AGAGTCCA', 'AGGATAGC']
#Indexes = ['ATTGCATA', 'TGGCGTAT', 'GCGATAGC', 'CGAATGCG', 'TTGTCAGG', 'AACGCACC', 'CCGGTAAC', 'GGCGATCA', 'GAAATCTA', 'TCAGTAGA', 'AGGTCCGA', 'CATGTATC']

#Make our dictionary that will hold our index pairs
IndexPairs = {} #This dictionary holds our expected indexes
IndexPairsList = 0 #Keeps count

IndexHop = {} #This dictionary holds our index hops
IndexHopList = 0 #Keeps count

IndexWithN = 0 #This is a count of any index pairs found with an N

NonIndexPairs = {} #This dictionary holds any non 'N' index pairs not part of our index library
NonIndexPairsList = 0 #Keeps count

Total_Reads=0 #Keeps count of total number of reads

#Open both the index files simultaneously and go line by line comparing each
#with open("Undetermined_S0_R2_001_trunc.fastq") as textfile1, open("Undetermined_S0_R3_001_trunc.fastq") as textfile2:
with open("R2.fastq") as textfile1, open("R3.fastq") as textfile2: 
    NR=0
    for x, y in zip(textfile1, textfile2):
        NR+=1
        if NR%4 == 2: #Looks at the sequence lines
            Total_Reads+=1
            x = x.strip() #Removes the \n
            y = y.strip()
            y = Complement(y) #Gets the reverse complement of the index
            #print(x)
            #print(y)
            #Checks to see if both indexes from our list of known
            if x in Indexes and y in Indexes:
                if x == y:
                    IndexPairsList+=1
                    if str(x+" "+y) in IndexPairs:
                        IndexPairs[x+" "+y]+=1
                    else:
                        IndexPairs[x+" "+y]=1
                elif x != y:
                    IndexHopList+=1
                    if str(x+" "+y) in IndexHop:
                        IndexHop[x+" "+y]+=1
                    else:
                        IndexHop[x+" "+y]=1
            #Checks to see if either of the indexes have an undefined 'N' character
            elif 'N' in x or 'N' in y:
                IndexWithN+=1
            #Assumes all remaining indexes are non undefined non library pairs, likely sequence errors
            else:
                #This should be non N characters but not from our index list
                NonIndexPairsList+=1
                if str(x+" "+y) in NonIndexPairs:
                    NonIndexPairs[x+" "+y]+=1
                else:
                    NonIndexPairs[x+" "+y]=1

#Get some stats on the run
print("The number of Proper Index Pairs observed are:",IndexPairsList,"Percent of total reads:",round((IndexPairsList/Total_Reads),4)*100)
print("The number of Index Hopping Events observed are:",IndexHopList,"Percent of total reads:",round((IndexHopList/Total_Reads),4)*100)
print("The number of Index pairs with at least one 'N' are:", IndexWithN,"Percent of total reads:",round((IndexWithN/Total_Reads),4)*100)
print("The number of Non Proper Non 'N' Indexes observed is:",NonIndexPairsList,"Percent of total reads:",round((NonIndexPairsList/Total_Reads),4)*100)
print("The total number of reads examined was",Total_Reads)

with open('ProperIndexPairsDic.txt', 'w') as f:
     f.write(str(IndexPairs))
with open('IndexHoppingDic.txt', 'w') as f:
     f.write(str(IndexHop))
with open('NonIndexNonUndefinedPairs.txt', 'w') as f:
     f.write(str(NonIndexPairs))

#print(IndexPairs)

# Code below was used to generate the quality score filtered index swaps

In [ ]:
#Index Hopping Problem

#Write a function to get the complement of the reverse index
def Complement(index):
    #First we make our index into a list so we can modify it
    index = list(index)
    for each in range(len(index)):
        #First we replace all the nucleotides to complements
        if index[each]=='N':
            index[each]='N'
        elif index[each]=='A':
            index[each]='T'
        elif index[each]=='T':
            index[each]='A'
        elif index[each]=='C':
            index[each]='G'
        elif index[each]=='G':
            index[each]='C'
    #Now we return it to a string
    index="".join(index)
    #Then we inverse the order
    index=index[::-1]
    return index

#To convert phred scores to numbers
def convert_phred(letter):
    """Converts a single character into a phred score"""
    QScore = ord(letter) - 33
    return QScore

#Establish our list of known indexes. THE FIRST IS THE REAL FILE, SECOND IS FOR TEST FILE, AND REVERSE COMPLEMENT
Indexes = ['GTAGCGTA', 'CGATCGAT', 'GATCAAGG', 'AACAGCGA', 'TAGCCATG', 'CGGTAATC', 'CTCTGGAT', 'TACCGGAT', 'CTAGCTCA', 'CACTTCAC', 'GCTACTCT', 'ACGATCAG', 'TATGGCAC', 'TGTTCCGT', 'GTCCTAAG', 'TCGACAAG', 'TCTTCGAC', 'ATCATGCG', 'ATCGTGGT', 'TCGAGAGT', 'TCGGATTC', 'GATCTTGC', 'AGAGTCCA', 'AGGATAGC']
#Indexes = ['ATTGCATA', 'TGGCGTAT', 'GCGATAGC', 'CGAATGCG', 'TTGTCAGG', 'AACGCACC', 'CCGGTAAC', 'GGCGATCA', 'GAAATCTA', 'TCAGTAGA', 'AGGTCCGA', 'CATGTATC']

#Make our dictionary that will hold our index pairs
IndexPairs = {} #This dictionary holds our expected indexes
IndexPairsList = 0 #Keeps count

IndexHop = {} #This dictionary holds our index hops
IndexHopList = 0 #Keeps count

IndexWithN = 0 #This is a count of any index pairs found with an N

NonIndexPairs = {} #This dictionary holds any non 'N' index pairs not part of our index library
NonIndexPairsList = 0 #Keeps count

Total_Reads=0 #Keeps count of total number of reads
Kept_reads=0 #Keeps track of all kept reads

#Open both the index files simultaneously and go line by line comparing each
#with open("Undetermined_S0_R2_001_trunc.fastq") as textfile1, open("Undetermined_S0_R3_001_trunc.fastq") as textfile2:
with open("R2.fastq") as textfile1, open("R3.fastq") as textfile2: 
    NR=0
    for x, y in zip(textfile1, textfile2):
        NR+=1
        if NR%4 == 0: #Looks at the quality scores
            #This first part will look at the quality scores and see if they are worth retaining
            current_read_FW = 0
            current_read_RV = 0
            x=x.strip()
            y=y.strip()
            #This loop adds up all the quality scores of the current line
            for i in range((len(x))):
                current_read_FW = current_read_FW + convert_phred(x[i])
                current_read_RV = current_read_RV + convert_phred(y[i])
            #Once we have all the quality scores we get the mean by dividing by the sequence length
            current_read_FW = current_read_FW/((len(x)))
            current_read_RV = current_read_RV/((len(y)))
            finalQS = (current_read_FW+current_read_RV)/2
            #The following line will retain the index pair if the QS is above or equal to 30, if so it will retain in
            if finalQS >= 30:
                Kept_reads+=1
                #Checks to see if both indexes from our list of known
                if FW in Indexes and RV in Indexes:
                    #If they are equal, it's a proper match
                    if FW == RV:
                        IndexPairsList+=1
                        if str(FW+"_"+RV) in IndexPairs:
                            IndexPairs[FW+"_"+RV]+=1
                        else:
                            IndexPairs[FW+"_"+RV]=1
                    #If not equal, but part of index list, then likely index hopping
                    elif FW != RV:
                        IndexHopList+=1
                        if str(FW+"_"+RV) in IndexHop:
                            IndexHop[FW+"_"+RV]+=1
                        else:
                            IndexHop[FW+"_"+RV]=1
                #Checks to see if either of the indexes have an undefined 'N' character
                elif 'N' in FW or 'N' in RV:
                    IndexWithN+=1
                #Assumes all remaining indexes are non undefined non library pairs, likely sequence errors
                else:
                    #This should be non N characters but not from our index list
                    NonIndexPairsList+=1
                    if str(FW+"_"+RV) in NonIndexPairs:
                        NonIndexPairs[FW+"_"+RV]+=1
                    else:
                        NonIndexPairs[FW+"_"+RV]=1
        if NR%4 == 2: #Looks at the sequence lines
            Total_Reads+=1            
            FW = x.strip() #Removes the \n
            RV = y.strip()
            RV = Complement(RV) #Gets the reverse complement of the index
            
#Get some stats on the run
print("The number of Proper Index Pairs observed are:",IndexPairsList,"Percent of total kept reads:",round((IndexPairsList/Kept_reads)*100,2))
print("The number of Index Hopping Events observed are:",IndexHopList,"Percent of total kept reads:",round((IndexHopList/Kept_reads)*100,2))
print("The number of Index pairs with at least one 'N' are:", IndexWithN,"Percent of total kept reads:",round((IndexWithN/Kept_reads)*100,2))
print("The number of Non Proper Non 'N' Indexes observed is:",NonIndexPairsList,"Percent of total kept reads:",round((NonIndexPairsList/Kept_reads)*100,2))
print("The total number of reads kept was ",Kept_reads,"Percent of reads kept:",round((Kept_reads/Total_Reads)*100,2))
print("The total number of reads examined was",Total_Reads)

#Write out our output files
with open('ProperIndexPairsDic.txt', 'w') as f:
     f.write(str(IndexPairs))
with open('IndexHoppingDic.txt', 'w') as f:
     f.write(str(IndexHop))
with open('NonIndexNonUndefinedPairs.txt', 'w') as f:
     f.write(str(NonIndexPairs))

# Bash code for post data clean up

In order to fix the dictionary outputs into a nice colon deliniated list for R, the following bash command is used:
```
cat R2readQS.txt | sed 's/{//' | sed 's/}//' | tr -d \'\" | tr ',' '\n' | sed 's/://' | sed -e '$a\' > myfile
```
In order to fix the list files with the bp QS scores, the following bash command was used:
```
cat R3bpQS.txt | tr '[' '_' | tr ']' '_' | sed 's/_//g' | sed 's/,//g' | sed 's/ /\n/g' | sed -e '$a\'
```

To take a look at the large dictionary files printer on unix the following command was used:
```
cat NonIndexNonUndefinedPairs.txt | sed 's/{//' | sed 's/}//' | tr -d \'\" | tr ',' '\n' | sort -t':' -rnk2 | head
```